

![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/healthcare/NER_HEALTHCARE_DE.ipynb)




# **Detect symptoms, treatments and other NERs in German**

To run this yourself, you will need to upload your license keys to the notebook. Otherwise, you can look at the example outputs at the bottom of the notebook. To upload license keys, open the file explorer on the left side of the screen and upload `workshop_license_keys.json` to the folder that opens.

## 1. Colab Setup

Import license keys

In [1]:
import os
import json

with open('/content/spark_nlp_for_healthcare.json', 'r') as f:
    license_keys = json.load(f)

license_keys.keys()

secret = license_keys['SECRET']
os.environ['SPARK_NLP_LICENSE'] = license_keys['SPARK_NLP_LICENSE']
os.environ['AWS_ACCESS_KEY_ID'] = license_keys['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY'] = license_keys['AWS_SECRET_ACCESS_KEY']
sparknlp_version = license_keys["PUBLIC_VERSION"]
jsl_version = license_keys["JSL_VERSION"]

print ('SparkNLP Version:', sparknlp_version)
print ('SparkNLP-JSL Version:', jsl_version)

SparkNLP Version: 2.6.0
SparkNLP-JSL Version: 2.6.0


Install dependencies

In [2]:
# Install Java
! apt-get update -qq
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
! java -version

# Install pyspark
! pip install --ignore-installed -q pyspark==2.4.4

# Install Spark NLP
! pip install --ignore-installed spark-nlp==$sparknlp_version
! python -m pip install --upgrade spark-nlp-jsl==$jsl_version --extra-index-url https://pypi.johnsnowlabs.com/$secret

openjdk version "11.0.8" 2020-07-14
OpenJDK Runtime Environment (build 11.0.8+10-post-Ubuntu-0ubuntu118.04.1)
OpenJDK 64-Bit Server VM (build 11.0.8+10-post-Ubuntu-0ubuntu118.04.1, mixed mode, sharing)
     |████████████████████████████████| 215.7MB 64kB/s 
     |████████████████████████████████| 204kB 42.3MB/s 
     |████████████████████████████████| 133kB 2.9MB/s 
Looking in indexes: https://pypi.org/simple, https://pypi.johnsnowlabs.com/2.6.0-8388813d58b67fa25bf9cf603393363af96dba16


Import dependencies into Python and start the Spark session

In [3]:
os.environ['JAVA_HOME'] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ['PATH'] = os.environ['JAVA_HOME'] + "/bin:" + os.environ['PATH']

import pandas as pd
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

import sparknlp
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.base import *
import sparknlp_jsl

spark = sparknlp_jsl.start(secret)

## 2. Construct the pipeline

For more details: https://github.com/JohnSnowLabs/spark-nlp-models#pretrained-models---spark-nlp-for-healthcare

In [4]:
document_assembler = DocumentAssembler() \
    .setInputCol('text')\
    .setOutputCol('document')

sentence_detector = SentenceDetector() \
    .setInputCols(['document'])\
    .setOutputCol('sentence')

tokenizer = Tokenizer()\
    .setInputCols(['sentence']) \
    .setOutputCol('token')

# German word embeddings
word_embeddings = WordEmbeddingsModel.pretrained('w2v_cc_300d','de', 'clinical/models') \
                    .setInputCols(["sentence", 'token'])\
                    .setOutputCol("embeddings")

# German NER model
clinical_ner = NerDLModel.pretrained('ner_healthcare','de', 'clinical/models') \
          .setInputCols(["sentence", "token", "embeddings"]) \
          .setOutputCol("ner")

ner_converter = NerConverter()\
    .setInputCols(['sentence', 'token', 'ner']) \
    .setOutputCol('ner_chunk')

nlp_pipeline = Pipeline(stages=[
    document_assembler, 
    sentence_detector,
    tokenizer,
    word_embeddings,
    clinical_ner,
    ner_converter])

empty_df = spark.createDataFrame([['']]).toDF("text")
pipeline_model = nlp_pipeline.fit(empty_df)
light_pipeline = LightPipeline(pipeline_model)

w2v_cc_300d download started this may take some time.
Approximate size to download 1.2 GB
[OK!]
ner_healthcare download started this may take some time.
Approximate size to download 14.1 MB
[OK!]


## 3. Create example inputs

In [7]:
# Enter examples as strings in this array
input_list = [
    """Ja das ist bekannt gewesen. Und die haben sich danach gerichtet. Hatte das Gefühl das er sich schon sehr wohl da gefühlt. Haben das auch von den Betreuern als Rückmeldung erhalten. Er ist zwar immer furchtbar aufgeregt wenn es dann los geht, aber wenn er dann da ist, freut er sich doch. Müssen zwar dann auch meist er sehr lange Autofahrt in Kauf nehmen da es das nicht in unserer Gegend gibt. Aber wenn er wenigstens daran eine Freude, warum nicht.
Leute, lest euch diesen Beitrag vollständig durch! Cosma und ich, wir haben unsere Erlebnisse gut dokumentiert. Wir waren am Anfang genauso ratlos, wie du (und noch andere, die hier dieselben Fragen stellem), die Diagnose ist für solche Sachen entweder Depressionen oder Burnout bzw. beides (wenn nichts Körperliches vorliegt). Manche haben noch zusätzlich Angstzustände oder Panikattacken. Die Diagnose sollte allerdings ein Neurologe/Psychiater stellen.""",
    """Das ist doch immer eine Überlegung wert. Bis vor kurzem war das sogar noch die Strategie ganzer Länder, u. a. GB. Also ich bin zum Besispiel einer von den süßen Opis, die derzeit bloß nicht an den Eiern gedrückt werden sollen. Ich würde mir tatsächlich eine Ansteckung mit einem einigermaßen moderaten Verlauf wünschen. Dann hätte ich, nach allem was man weiß, u. a. von Rhesusaffen, eine Grundimmunität und könnte rausgehen und meinen Mitmenschen unter die Arme greifen. Ich bin halt alt, aber net unbedingt schwach. Rüstiger Greis, sozusagen""",
]

## 4. Use the pipeline to create outputs

In [8]:
df = spark.createDataFrame(pd.DataFrame({"text": input_list}))
result = pipeline_model.transform(df)

## 5. Visualize results

Visualize outputs as data frame

In [9]:
exploded = F.explode(F.arrays_zip('ner_chunk.result', 'ner_chunk.metadata'))
select_expression_0 = F.expr("cols['0']").alias("chunk")
select_expression_1 = F.expr("cols['1']['entity']").alias("ner_label")
result.select(exploded.alias("cols")) \
    .select(select_expression_0, select_expression_1).show(truncate=False)
result = result.toPandas()

+--------------+-----------------+
|chunk         |ner_label        |
+--------------+-----------------+
|Cosma         |MEDICATION       |
|gut           |STATE_OF_HEALTH  |
|am Anfang     |TIME_INFORMATION |
|Diagnose      |DIAGLAB_PROCEDURE|
|Depressionen  |MEDICAL_CONDITION|
|Burnout       |MEDICAL_CONDITION|
|Angstzustände |MEDICAL_CONDITION|
|Panikattacken |MEDICAL_CONDITION|
|Diagnose      |DIAGLAB_PROCEDURE|
|Bis vor kurzem|TIME_INFORMATION |
|Ansteckung    |MEDICAL_CONDITION|
|Rhesusaffen   |MEDICATION       |
|Grundimmunität|TREATMENT        |
|Arme          |BODY_PART        |
|schwach       |MEASUREMENT      |
|Greis         |MEDICAL_CONDITION|
+--------------+-----------------+



Functions to display outputs as HTML

In [10]:
from IPython.display import HTML, display
import random

def get_color():
    r = lambda: random.randint(128,255)
    return "#%02x%02x%02x" % (r(), r(), r())

def annotation_to_html(full_annotation):
    ner_chunks = full_annotation[0]['ner_chunk']
    text = full_annotation[0]['document'][0].result
    label_color = {}
    for chunk in ner_chunks:
        label_color[chunk.metadata['entity']] = get_color()

    html_output = "<div>"
    pos = 0

    for n in ner_chunks:
        if pos < n.begin and pos < len(text):
            html_output += f"<span class=\"others\">{text[pos:n.begin]}</span>"
        pos = n.end + 1
        html_output += f"<span class=\"entity-wrapper\" style=\"color: black; background-color: {label_color[n.metadata['entity']]}\"> <span class=\"entity-name\">{n.result}</span> <span class=\"entity-type\">[{n.metadata['entity']}]</span></span>"

    if pos < len(text):
        html_output += f"<span class=\"others\">{text[pos:]}</span>"

    html_output += "</div>"
    display(HTML(html_output))

Display example outputs as HTML

In [11]:
for example in input_list:
    annotation_to_html(light_pipeline.fullAnnotate(example))